## <font color=blue>Contents<font/>

#########################################################################################################

1. **Jupyter setup <br>**
2. **Data Load <br>**
3. **Exploratory Data Analysis <br>**
    * **Identify Variables and Variable Types and Clean**
    * **Derive Variables**
    * **Identify and Handle Outliers**
    * **Inspect Distributions**
    * **Remove Outlier Rows and Rerun Distribution Plots** <br><br>
4. **Regression Model Builds**

#########################################################################################################

## <font color=blue>Jupyter Setup<font/>

###  Import required libraries:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
import math
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 50)

## <font color=blue>Data Load<font/>

### Load data:

In [3]:
orig_df = pd.read_csv('kc_house_data.csv')

FileNotFoundError: [Errno 2] File b'kc_house_data.csv' does not exist: b'kc_house_data.csv'

In [ ]:
df = orig_df.copy()

### Preview data:

In [ ]:
df.head()

## <font color=blue>Exploratory Data Analysis</font>

### Identify variables and variable Types:

In [ ]:
df.info()

Upon inital inspection:

1. Three variables have null values: **`waterfront`**, **`view`** and **`yr_renovated`**
2. **`date`** looks to be a string variable instead of datetime

###############################################################

### Inspect each variable individually, looking for any further integrity issues

###  <font color=green>id</font>

Would it be possible to use **`id`** as the index column?

In [ ]:
len(df['id'].unique())

No, as there are duplicate ids within the column which indicates that the same property has been sold multiple times within the time frame of the dataset.

A follow up question is whether it would be worth creating a derived variable that flags those properties that have been sold multiple times to be used as possible input variable fo rthe regression model?

In [ ]:
id_vc = pd.DataFrame(df['id'].value_counts())  # Check how many times properties appear within the dataset
id_vc = id_vc.reset_index()
id_vc.columns

number_of_unique_ids = len(id_vc.loc[id_vc['id'] > 1].sort_values(by = 'id', ascending = False))

print('{}'.format(round((number_of_unique_ids / len(id_vc))*100, 2)) + '% of properties were sold multiple times')

Properties sold multiple times only account for 0.82% of all unique properties within the dataset and therefore there is no real value in deriving a variable to flag 'number of times sold'

### <br><font color=green>date</font>

In [ ]:
df['date'].describe()
type(df['date'][0])

df['date'].sort_values(ascending=True).head()

the **`date`** variable is stored as a str object and needs to be converted to **datetime** format

In [ ]:
df['date'] = pd.to_datetime(df['date'])

Verifying the modification has worked:

In [ ]:
df['date'].sort_values(ascending=True).head()

In [ ]:
df['date'].sort_values(ascending=False).head()

In [ ]:
type(df['date'][0])

### <font color=green>price</font>

In [ ]:
df['price'].describe()

In [ ]:
type(df['price'][0])

**`price`** variable type is fine

### <font color=green>bedrooms</font>

In [ ]:
df['bedrooms'].describe()

In [ ]:
print(type(df['bedrooms'][0]))
df['bedrooms'].value_counts()

**`bedrooms`** variable is of correct type but there does seem to be some extreme values: 33, 11 and 10

The variable is also clearly a categorical variable.

Outliers will be dealt with later

### <font color=green>bathrooms</font>

In [ ]:
df['bathrooms'].describe()

In [ ]:
print(type(df['bathrooms'][0]))
df['bathrooms'].value_counts()

**`bathrooms`** variable is of correct type but there could be possible extreme outliers of bathrooms per bedroom > 6

This variable can potentially be considered as a categorical variable

### <font color=green>sqft_living</font>

In [ ]:
df['sqft_living'].describe()

In [ ]:
print(type(df['sqft_living'][0]))
df['sqft_living'].value_counts()

**`sqft_living`** variable is fine

### <font color=green>sqft_lot</font>

In [ ]:
df['sqft_lot'].describe()

In [ ]:
print(type(df['sqft_lot'][0]))
df['sqft_lot'].value_counts()

**`sqft_lot`** variable is of correct type but there does seem to be some extreme values: 1,651,359 sq ft.<br>

Outliers will be dealt with later

### <font color=green>floors</font>

In [ ]:
df['floors'].describe()

In [ ]:
type(df['floors'][0])
df['floors'].value_counts()

**`floors`** is of correct type

This variable is a categorical variable

### <font color=green>waterfront</font>

In [ ]:
df['waterfront'].describe()

In [ ]:
df['waterfront'].isna().sum()

the **`waterfront`** variable has 2,376 missing values

This variable is a categorical variable

In [ ]:
print(type(df['waterfront'][0]))

#df['waterfront'] = df['waterfront'].astype('int64')

df['waterfront'].value_counts()

the variable is of correct boolean type

the missing will need to be imputed with a placeholder value: **9.0**

In [ ]:
df['waterfront'] = df['waterfront'].fillna(9.0)

Verifying the imputation has worked:

In [ ]:
df['waterfront'].value_counts()

In [ ]:
df['waterfront'] = df['waterfront'].astype('int64')
df['waterfront'].value_counts()

### <font color=green>view</font>

In [ ]:
df['view'].describe()

In [ ]:
df['view'].isna().sum()

the **`view`** variable has 63 missing values

In [ ]:
type(df['view'][0])

the variable is of correct numerical type

the missing will need to be imputed with a placeholder value: median of non-null **`view`**

In [ ]:
df['view'].value_counts()

In [ ]:
view_median = df['view'].loc[df['view'].isna() == False].median()
print(view_median)

In [ ]:
df['view'] = df['view'].fillna(int(view_median))

Verifying the imputation has worked:

In [ ]:
df['view'] = df['view'].astype('int64')

df['view'].value_counts()

In [ ]:
df['view'].describe()

This variable is a categorical variable

### <font color=green>condition</font>

In [ ]:
print(df['condition'].describe(),'\n')
print(type(df['condition'][0]))
df['condition'].value_counts()

**`condition`** is of correct type

This variable is a categorical variable

### <font color=green>grade</font>

In [ ]:
print(df['grade'].describe(), '\n')
print(type(df['grade'][0]))
df['grade'].value_counts()

**`grade`** is of correct type

This variable is a categorical variable

### <font color=green>sqft_above</font>

In [ ]:
print(df['sqft_above'].describe(), '\n')
print(type(df['sqft_above'][0]))
df['sqft_above'].value_counts()

### <font color=green>sqft_basement</font>

In [ ]:
print(df['sqft_basement'].describe(), '\n')
print(type(df['sqft_basement'][0]))
df['sqft_basement'].value_counts()

There is a random '?' value within the **`sqft_basement`** variable and needs to be treated as a missing value but since we want to convert the variable to of numerical type we need to replace the '?' with a numerical missing value placeholder.

First need to check the median value for all values != '?'

In [ ]:
sqft_basement_median = df['sqft_basement'].loc[df['sqft_basement'] != '?'].median()
print(sqft_basement_median)

replace '?' with '0.0':

In [ ]:
df['sqft_basement'] = df['sqft_basement'].replace('?',str(sqft_basement_median))

Verify imputation:

In [ ]:
df['sqft_basement'].value_counts()

<br> The **`sqft_basement`** variable is stored as a str needs to be converted into a integer format

In [ ]:
df['sqft_basement'] = df['sqft_basement'].apply(lambda x: float(x)).astype('int64')

Verify modification:

In [ ]:
print(df['sqft_basement'].describe(), '\n')
print(df['sqft_basement'].isna().sum(), '\n')
df['sqft_basement'].head()

### <font color=green>yr_built</font>

In [ ]:
print(df['yr_built'].describe(), '\n')
print(type(df['yr_built'][0]))
df['yr_built'].value_counts()

Will be usefule to bin this variable into decades.

which will also make it a categorical variable.

Will be done later on

### <font color=green>yr_renovated</font>

In [ ]:
print(df['yr_renovated'].describe(), '\n')
print(type(df['yr_renovated'][0]))
df['yr_renovated'].value_counts()

* Missing values seem to exist
* a value of 0.0 is assumed to mean not renovated
* Categorical variable

In [ ]:
df['yr_renovated'].isna().sum()

3,842 missing values need to be imputed

First need to check whether 2050.0 will be a safe choice for a missing value placeholder:

In [ ]:
sorted(df['yr_renovated'].loc[df['yr_renovated'] != 0.0].unique(), reverse = True)

2050 will be a safe choice to represent missing values:

In [ ]:
df['yr_renovated'] = df['yr_renovated'].fillna(2050.0)
df['yr_renovated'] = df['yr_renovated'].astype('int64')
# verify imputation
df['yr_renovated'].value_counts()

### <font color=green>zipcode</font>

In [ ]:
print(df['zipcode'].describe(), '\n')
print(df['zipcode'].head(), '\n')
print(type(df['zipcode'][0]), '\n')
print(df['zipcode'].value_counts(), '\n')
df['zipcode'].isna().sum()

**`zipcode`** is of correct type

#### <font color=green>lat</font>

In [ ]:
print(df['lat'].describe(), '\n')
print(df['lat'].head(), '\n')
print(type(df['lat'][0]))

**`lat`** is of correct type

### <font color=green>long</font>

In [ ]:
print(df['long'].describe(), '\n')
print(df['long'].head(), '\n')
print(type(df['long'][0]))

**`long`** is of correct type

### <font color=green>sqft_living15</font>

In [ ]:
print(df['sqft_living15'].describe(), '\n')
print(df['sqft_living15'].head(), '\n')
print(type(df['sqft_living15'][0]), '\n')
df['sqft_living15'].value_counts()

### <font color=green>sqft_lot15</font>

In [ ]:
print(df['sqft_lot15'].describe(), '\n')
print(df['sqft_lot15'].head(), '\n')
print(type(df['sqft_lot15'][0]), '\n')
df['sqft_lot15'].value_counts()

This is the original structure of the dataframe:

<img src="orig_df_info.png" width = "400">

and this is the new structure after cleaning process cleaning:

##### <font color=red>Cleaned Dataset Formats</font>

In [ ]:
df.info()

### Derive Variables

#### `day_of_week`, `month`, `quarter` derived from `date`

It would be useful to explor whether the time of year or week has any impact on the price of a property. THerefore the day, month and quarter will need to be extracted from the **`date`** column

In [ ]:
#   Useful to extract the day, month and quarter from the date variable:

df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.bar(df['date'].value_counts().index, df['date'].value_counts());

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.bar(df['day_of_week'].value_counts().index, df['day_of_week'].value_counts());

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.bar(df['month'].value_counts().index, df['month'].value_counts());

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.bar(df['quarter'].value_counts().index, df['quarter'].value_counts());

#### `distance_from_town_centre` derived from `long` and `lat`

Using a fixed longitude and latitude coordinates of [-122.3341, 47.6106] to represent the city centre of Seattle. We will derive a new variable that calculates the distance from the property to the city centre.

In [ ]:
from math import sin, cos, sqrt, atan2, radians

In [ ]:

def distance_calc(long_lat, city_centre_long_lat = [-122.3341, 47.6106]):
    
    # approximate radius of earth in km
    R = 6373.0
    city_centre_lon = radians(city_centre_long_lat[0])
    city_centre_lat = radians(city_centre_long_lat[1])
    
    addres_lon = radians(float(long_lat.split(',')[0]))
    address_lat = radians(float(long_lat.split(',')[1]))

    dlon = city_centre_lon - addres_lon
    dlat = city_centre_lat - address_lat

    a = sin(dlat / 2)**2 + cos(city_centre_lat) * cos(address_lat) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    
    return distance

In [ ]:
df['long_lat_coords'] = df["long"].map(str) + "," + df["lat"].map(str)

In [ ]:
df['dist_from_city_centre'] = df['long_lat_coords'].apply(lambda x: distance_calc(x))

In [ ]:
df.loc[:,['zipcode','long', 'lat', 'dist_from_city_centre']].head()

In [ ]:
df = df.loc[:,df.columns != 'long_lat_coords'] # Drop this column as it is not needed for our analysis

#### `livingsqft_lotsqft_ratio` derived from `sqft_living` and `sqft_lot`

It has been noted that there are instances where the size of the living space (**`sqft_living`**) is considerably smaller than the size of the lot **`sqft_lot`**. So it will be useful to create a metric that measures the ratio of the living to lot size.

In [ ]:
df['livingsqft_lotsqft_ratio'] = df['sqft_living']/df['sqft_lot']

In [ ]:
df.loc[:, ['sqft_living', 'sqft_lot', 'livingsqft_lotsqft_ratio']].head()

#### `yrs_since_renovation` derived from `yr_renovated`

Create a measure of **`yr_renovated`** that is a standardised measure regardless of the year in which the property was sold.

In [ ]:
def roundown(x):
    return int(math.floor(x / 10.0)) * 10

def yrs_since_renov(row):
    if (row['yr_renovated'] not in (0, 2050)) and (row['date'].year > row['yr_renovated']):
        return roundown(row['date'].year - row['yr_renovated'])
    elif row['yr_renovated'] == 2050:
        return -2
    else:
        return -1
    
df['yrs_since_renovation'] = df.apply(yrs_since_renov, axis=1)

In [ ]:
print(df['yrs_since_renovation'].value_counts())

df.head()

#### `decade_built` derived from `yr_built`

Will be useful to bin the **`yr_built`** variable for exploration purposes

In [ ]:
df['decade_built'] = df['yr_built'].apply(lambda x: roundown(x))

In [ ]:
df['decade_built'].value_counts()

#### Create dummy alternatives of categorical variables

Many of the independent variables are categorical by nature and so it is required to convert the to dummy vasriables using the one-hot encoding method

In [ ]:
df.loc[:,['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view', 'condition', 'grade', 'decade_built', 'yrs_since_renovation']] = df.loc[:,['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view', 'condition', 'grade', 'decade_built', 'yrs_since_renovation']].astype('int64')

In [ ]:
cat_var_dict = {'empty': None}

for i in ['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view', 'condition', 'grade', 'decade_built', 'yrs_since_renovation']:
  
    cat_var_dict[str(i)+'_dummies'] = pd.get_dummies(df[str(i)], prefix='d_' + str(i)[:5], drop_first=True)
    


In [ ]:
list(cat_var_dict.keys())[1:]

In [ ]:
df = pd.concat([df, cat_var_dict['bedrooms_dummies'], cat_var_dict['bathrooms_dummies'], cat_var_dict['floors_dummies'], cat_var_dict['waterfront_dummies'], cat_var_dict['view_dummies'], cat_var_dict['condition_dummies'],
             cat_var_dict['grade_dummies'], cat_var_dict['decade_built_dummies'], cat_var_dict['yrs_since_renovation_dummies']], axis=1)

In [ ]:
df.head()

### Identify and Handle Outliers

In [ ]:
list(df.columns)

In [ ]:
df_no_dummies = df.loc[:,['id',
 'date',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'month',
 'quarter',
 'dist_from_city_centre',
 'livingsqft_lotsqft_ratio',
 'yrs_since_renovation',
 'decade_built']]

In [ ]:
fig2, axes2 = plt.subplots(5, 5, figsize = (20,40))
axes2.shape




for n in range(1,24):
    
    i = df_no_dummies.dtypes.index[n-1]
    
    row = (n-1)//5
    col = (n-1)%5
    
    if str(type(df_no_dummies[str(i)][0])) in ["<class 'numpy.float64'>", "<class 'numpy.int64'>"]:
        ax = axes2[row][col]
        ax.boxplot(df[str(i)])
        ax.set_title('{}'.format(i))
    else:   
        ax = axes2[row][col]
        ax.set_title('{}'.format(i))
        continue

Initial observations:

* **`bedrooms`**, **`sqft_lot`**, **`sqft_basement`**, **`sqft_lot15`** have clear extreme outliers which require removing/imputation
* **`bathrooms`** may have possible outliers of >6
* boxplots for **`view`**, **`yr_renovated`** have been distorted due to the previous step of imputing values for the missing observations with 9999.0. The box plot will have to be re-run without these imputed observations


In [ ]:
fig2, axes2 = plt.subplots(1, 2, figsize = (10,10))
axes2.shape

col = (-1)
for n in range(1,27):
    
    i = df.dtypes.index[n-1]
    
    row = 1
    
    
    if i in ['view']:
        
        col += 1
        a = [x for x in df[str(i)].value_counts().index if x != 9999.0]
        b = [df[str(i)].value_counts().ix[x] for x in df[str(i)].value_counts().index if x != 9999.0]
        

        ax = axes2[col]
        ax.boxplot(a)
        ax.set_title('{}'.format(i))
        
    elif i in ['yr_renovated']:
        
        col += 1
        a = [x for x in df[str(i)].value_counts().index if (x != 9999.0) and (x != 0.0)]
        b = [df[str(i)].value_counts().ix[x] for x in df[str(i)].value_counts().index if x != 9999.0]
        
       
        ax = axes2[col]
        ax.boxplot(a)
        ax.set_title('{}'.format(i));
    else:   
        continue

### Inspect Distributions

In [ ]:
fig, axes = plt.subplots(9, 3, figsize = (20,40))
axes.shape


for n in range(1,27):
    
    i = df.dtypes.index[n-1]
    
    row = (n-1)//3
    col = (n-1)%3
    
    if len(df[str(i)].unique()) > 50:
        #continue
        ax = axes[row][col]
        ax.set_title('{}'.format(i))
        ax.hist(df[str(i)], bins = 100)
        continue
    
    a = [x for x in df[str(i)].value_counts().index if x != 9999.0]
    b = [df[str(i)].value_counts().ix[x] for x in df[str(i)].value_counts().index if x != 9999.0]
        
    ax = axes[row][col]
    ax.bar(a, b)
    ax.set_title('{}'.format(i))  

In [ ]:
outl_vars = ['bedrooms', 'sqft_lot', 'sqft_basement', 'sqft_lot15', 'bathrooms']
def remove_outliers_and_plot_bars(list_of_vars):
    
    fig2, axes2 = plt.subplots(1, 5, figsize = (20,5))

    col = (-1)
    
    for i in list_of_vars:
        
            
        col += 1
    
        pcntle = np.percentile(df[str(i)], 75)
        
        
        
        if len(df[str(i)].unique()) > 50:
            ax = axes2[col]
            ax.hist(df[str(i)].loc[df[str(i)] < pcntle], bins = 30)
            ax.set_title('{}'.format(i))
    
            continue
    
        else:

            a = [x for x in df[str(i)].value_counts().index if x < pcntle]
            b = [df[str(i)].value_counts().ix[x] for x in df[str(i)].value_counts().index if x < pcntle]
            
            ax = axes2[col]
            ax.bar(a, b)
            ax.set_title('{}'.format(i))
    
    col = (-1)
        
remove_outliers_and_plot_bars(outl_vars)

In [ ]:
fig3, axes3 = plt.subplots(9, 3, figsize = (25,40))
axes3.shape


for n in range(1,27):
    
    i = df.dtypes.index[n-1]
    
    row = (n-1)//3
    col = (n-1)%3
    
    if str(type(df[str(i)][0])) in ["<class 'numpy.float64'>", "<class 'numpy.int64'>"]:
        
        if (len(df[str(i)].unique()) > 12) and (str(i) not in ['bathrooms']):
        
            ax = axes3[row][col]
            ax.scatter(df[str(i)], df['price'])
            ax.set_title('{}'.format(i) + ' vs price')
            
        
        else:
            ax = axes3[row][col]
            sns.boxplot(str(i), 'price', data=df, ax=ax)
            ax.set_title('{}'.format(i) + ' vs price')
    else:   
        ax = axes3[row][col]
        ax.set_title('{}'.format(i))
        continue

In [ ]:
outl_vars = ['sqft_living', 'sqft_lot', 'sqft_basement', 'sqft_lot15', 'sqft_living15'
             , 'sqft_above', 'yr_built', 'dist_from_city_centre'
             ,'livingsqft_lotsqft_ratio']


for i in outl_vars:
    
    df[(str(i) + '_bins')] = pd.cut(df[str(i)], 10, labels = list(range(1,11)))
    
    


def remove_outliers_and_plot_grouped_boxes(list_of_vars):
    
    fig2, axes2 = plt.subplots(3, 3, figsize = (20,20))

   
    n = (-1)
    for i in list_of_vars:
        
        n += 1
        
        row = (n)//3
        col = (n)%3
        
        ax = axes2[row][col]
        sns.boxplot((str(i) + '_bins'), 'price', data=df, ax=ax)
        ax.set_title('{}'.format(str(i)) + ' vs price')
    

    n = (-1)  
        
remove_outliers_and_plot_grouped_boxes(outl_vars)

In [ ]:
sns.boxplot('yrs_since_renovation', 'price', data=df).set_title('yr_renovated vs price')

**Initial observations:**

The following variables show some sort of correlation upon price (observed either in the scatter plots or the grouped box-plots):

* **`sqft_living`**
* **`sqft_lot`**
* **`bedrooms`**
* **`bathrooms`**
* **`floors`**
* **`condition`**
* **`sqft_living15`**
* **`grade`**
* **`dist_from_city_centre`**
* **`livingsqft_lotsqft_ratio`**
* **`yrs_since_renovation`**

just to confirm by calculating Pearson's Correlation for the above variables against **`price`**

In [ ]:
from scipy.stats import pearsonr


for i in ['sqft_living', 'sqft_lot', 'bedrooms', 'bathrooms', 'floors', 'condition', 'sqft_living15', 'grade', 'dist_from_city_centre', 'livingsqft_lotsqft_ratio', 'yrs_since_renovation']:

    print('{} :'.format(i) + str(round(pearsonr(df[i], df['price'])[0], 2)))

Based on Pearson's correlation the following variables show the strongest correlation with price:

* **`sqft_living`**   => 0.7
* **`bathrooms`**     => 0.51
* **`sqft_living15`** => 0.59
* **`grade`**         => 0.67

### Remove Outliers

The next steps will remove outliers from the dataset.

In [ ]:
df = df.loc[df['bedrooms'] < 33] # Obvious outlier from the scatter plot

In [ ]:
df = df.loc[df['sqft_lot'] < 1651359] # Obvious outlier from the scatter plot

In [ ]:
df_without_outliers = pd.DataFrame(None)

# below loop will loop through all the variables that have previously been identified as having upper outliers and will
# flag the observations to which the outliers belong in the dataset (only upper outliers)

for i in ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot'
          ,'floors', 'sqft_above','sqft_basement'
          ,'sqft_living15', 'sqft_lot15']:
    
    data_mean, data_std = np.mean(df[str(i)]), np.std(df[str(i)])
    # identify outliers
    cut_off = data_std * 3
    upper = data_mean + cut_off
    df_without_outliers[str(i) + '_outlier_flag'] = (df[str(i)] >= upper)
        
        
df_without_outliers = df_without_outliers.replace(True,np.nan).dropna(axis = 0, how = 'any')

In [ ]:
df_without_outliers

In [ ]:
indexes_to_keep = df_without_outliers.index

print(indexes_to_keep)
len(indexes_to_keep)


# highlight observations that do not have any outliers across any columns

In [ ]:
df_without_outlier_rows = df.ix[indexes_to_keep]

len(df_without_outlier_rows)

In [ ]:
df_without_outlier_rows.columns

Rerun exploratory scatter plots and boxplots from the previous section to compare how the correlations differ with outliers removed

In [ ]:
fig3, axes3 = plt.subplots(7, 4, figsize = (25,40))
axes3.shape


for n in range(1,29):
    
    i = df_without_outlier_rows.dtypes.index[n-1]
    
    row = (n-1)//4
    col = (n-1)%4
    
    if str(type(df_without_outlier_rows[str(i)][0])) in ["<class 'numpy.float64'>", "<class 'numpy.int64'>"]:
        
        if (len(df_without_outlier_rows[str(i)].unique()) > 12) and (str(i) not in ['bathrooms']):
        
            ax = axes3[row][col]
            ax.scatter(df_without_outlier_rows[str(i)], df_without_outlier_rows['price'])
            ax.set_title('{}'.format(i) + ' vs price')
            
        
        else:
            ax = axes3[row][col]
            sns.boxplot(str(i), 'price', data=df_without_outlier_rows, ax=ax)
            ax.set_title('{}'.format(i) + ' vs price')
    else:   
        ax = axes3[row][col]
        ax.set_title('{}'.format(i))
        continue

In [ ]:
outl_vars = ['sqft_lot', 'sqft_basement', 'sqft_lot15', 'sqft_living15'
             ,'sqft_above', 'yr_built', 'dist_from_city_centre'
             ,'livingsqft_lotsqft_ratio']


for i in outl_vars:
    
    df_without_outlier_rows[(str(i) + '_bins')] = pd.cut(df_without_outlier_rows[str(i)], 10, labels = list(range(1,11)))
    
    


def remove_outliers_and_plot_grouped_boxes(list_of_vars):
    
    fig2, axes2 = plt.subplots(3, 3, figsize = (20,40))

   
    n = (-1)
    for i in list_of_vars:
        
        n += 1
        
        row = (n)//3
        col = (n)%3
        
        ax = axes2[row][col]
        sns.boxplot((str(i) + '_bins'), 'price', data=df_without_outlier_rows, ax=ax)
        ax.set_title('{}'.format(str(i)) + ' vs price')
    

    n = (-1)  
        
remove_outliers_and_plot_grouped_boxes(outl_vars)

In [ ]:
sns.boxplot('yrs_since_renovation', 'price', data=df_without_outlier_rows).set_title('yr_renovated vs price')

In [ ]:
for i in ['sqft_living', 'sqft_lot', 'bedrooms', 'bathrooms', 'floors', 'condition', 'sqft_living15', 'grade', 'dist_from_city_centre', 'livingsqft_lotsqft_ratio', 'yrs_since_renovation', 'view']:

    print('{} :'.format(i) + str(round(pearsonr(df_without_outlier_rows[i], df_without_outlier_rows['price'])[0], 2)))

Compare pearson's correlation before outliers removed and after:

* **`sqft_living`**&ensp;&ensp;&emsp;&emsp;&emsp;=> 0.7,&emsp;&emsp;0.62
* **`bathrooms`**&emsp;&emsp;&emsp;&emsp;&emsp;=> 0.51,&emsp;&emsp;0.4
* **`sqft_living15`**&emsp;&emsp;=> 0.59,&emsp;&emsp;0.55
* **`grade`**&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;=> 0.67,&emsp;&emsp;0.64

No real change in correlation scores except with **`sqft_living`** and **`bathrooms`**

In [ ]:
df_without_outlier_rows.head()

## <font color=blue>Regression Model Build<font/>

The aim is to create four regression models from the dataset (with outliers removed):

* 1. One master model that seeks to identify the variables that most impact the price of the property. This model will use the variables highlighted as having the greatest correlation with **`price`** <br><br>
* 2. three sub models that seek to identify the relative impact of specific variables upon **`price`** in order to answer the following questions:<br><br>

    * Which structural features have the largest impact on price?
    * Does the cosmetic condition affect price?
    * Does the size of property and size of land affect price?

create three datasets for each question:<br>
    
    1) Which structural features make the largest impact on price?
    2) Does cosmetic condition affect price?
    3) Does size of property and land affect price?

Will be using the dummy converted variables instead of the original categorical variables for **`bathrooms`**, **`bedrooms`**, **`yrs_since_renovation`**, **`grade`** and **`condition`**

In [ ]:
df_without_outlier_rows.columns

In [ ]:
df_without_outlier_rows = df_without_outlier_rows.rename(columns={"d_yrs_s_-1": "d_yrs_s_minus1"}) 
# clean up a naming convention in one specific variable

In [ ]:
df_without_outlier_rows.head()

Create the 4 datasets to input into our regression model builds:

In [ ]:
df_overall = df_without_outlier_rows.loc[:,['price', 'd_bathr_1',
       'd_bathr_2', 'd_bathr_3', 'd_grade_4',
       'd_grade_5', 'd_grade_6', 'd_grade_7', 'd_grade_8', 'd_grade_9',
       'd_grade_10', 'd_grade_11', 'd_grade_12', 'sqft_living', 'sqft_living15']]


df_structural = df_without_outlier_rows.loc[:,['price', 'd_bedro_2', 'd_bedro_3',
       'd_bedro_4', 'd_bedro_5', 'd_bedro_6', 'd_bathr_1',
       'd_bathr_2', 'd_bathr_3', 'd_yrs_s_minus1', 'd_yrs_s_0', 'd_yrs_s_10',
       'd_yrs_s_20', 'd_yrs_s_30', 'd_yrs_s_40', 'd_yrs_s_50', 'd_yrs_s_60',
       'd_yrs_s_70', 'd_yrs_s_80']]

df_cosmetic = df_without_outlier_rows.loc[:,['price', 'd_condi_2', 'd_condi_3', 'd_condi_4', 'd_condi_5', 'd_yrs_s_minus1', 'd_yrs_s_0', 'd_yrs_s_10',
       'd_yrs_s_20', 'd_yrs_s_30', 'd_yrs_s_40', 'd_yrs_s_50', 'd_yrs_s_60',
       'd_yrs_s_70', 'd_yrs_s_80']]

df_size = df_without_outlier_rows.loc[:,['price', 'sqft_living', 'sqft_lot', 'livingsqft_lotsqft_ratio']]


In [ ]:
df_overall.head()

check for multicollinearity between predictor variables:

In [ ]:
a = df_without_outlier_rows.loc[:,['sqft_living', 'sqft_living15', 'bathrooms', 'grade']]

pd.plotting.scatter_matrix(a,figsize  = [15, 15]);
plt.show()

a.corr()

In [ ]:
sns.heatmap(a.corr(), center=0);

Need to exclude **`sqft_living`** from the regrrssionmodel build as it is highly correlated with the **`sqft_living15`**, **`bathrooms`**, **`grade`**

In [ ]:
outcome = 'price'
predictors = df_overall.drop(['price', 'sqft_living', 'd_grade_4',        # build model on just 'number of bathrooms'
       'd_grade_5', 'd_grade_6', 'd_grade_7', 'd_grade_8', 'd_grade_9',
       'd_grade_10', 'd_grade_11', 'd_grade_12', 'sqft_living15'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df_overall).fit()
model.summary()

In [ ]:
outcome = 'price'
predictors = df_overall.drop(['price', 'sqft_living', 'd_grade_4',        # build model on 'number of bathrooms'
       'd_grade_5', 'd_grade_6', 'd_grade_7', 'd_grade_8', 'd_grade_9',   # and 'sqft_living15'
       'd_grade_10', 'd_grade_11', 'd_grade_12'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df_overall).fit()
model.summary()

In [ ]:
outcome = 'price'
predictors = df_overall.drop(['price', 'sqft_living'],                   # build model on number of bathrooms,
             axis=1)                                                     # sqft_living15 and grade
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df_overall).fit()
model.summary()

R-squared = **0.46**

#### Question 1 - Which structural features make the largest impact on price?

In [ ]:
df_structural.columns

In [ ]:
# build model on 'number of bedrooms' only

outcome = 'price'
predictors = df_structural.drop(['price', 'd_bathr_1', 'd_bathr_2', 'd_bathr_3', 'd_yrs_s_minus1',
       'd_yrs_s_0', 'd_yrs_s_10', 'd_yrs_s_20', 'd_yrs_s_30', 'd_yrs_s_40',
       'd_yrs_s_50', 'd_yrs_s_60', 'd_yrs_s_70', 'd_yrs_s_80'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df_structural).fit()
model.summary()

In [ ]:
# build model on 'number of bedrroms' and 'number of bathrooms'

outcome = 'price'
predictors = df_structural.drop(['price', 'd_yrs_s_minus1',
       'd_yrs_s_0', 'd_yrs_s_10', 'd_yrs_s_20', 'd_yrs_s_30', 'd_yrs_s_40',
       'd_yrs_s_50', 'd_yrs_s_60', 'd_yrs_s_70', 'd_yrs_s_80'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df_structural).fit()
model.summary()

In [ ]:
# build model on 'number of bedrooms', 'number of bathrooms' and 'yrs since renovated'

outcome = 'price'
predictors = df_structural.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df_structural).fit()
model.summary()

R-squared = **0.2**

making structural changes to a property such as adding a bathroom(s), bedroom(s) or simplty renovating only accounts for 20% in the change in price of a property

#### Question 2 - How much impact does cosmetic condition have on price?

In [ ]:
df_cosmetic.columns

In [ ]:
# build model on 'condition' only

outcome = 'price'
predictors = df_cosmetic.drop(['price',
       'd_yrs_s_minus1', 'd_yrs_s_0', 'd_yrs_s_10', 'd_yrs_s_20', 'd_yrs_s_30',
       'd_yrs_s_40', 'd_yrs_s_50', 'd_yrs_s_60', 'd_yrs_s_70', 'd_yrs_s_80'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df_cosmetic).fit()
model.summary()

In [ ]:
# build model on 'condition' and 'yrs since renovation'

outcome = 'price'
predictors = df_cosmetic.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df_cosmetic).fit()
model.summary()

R-squared = **0.02**

making cosmetic improvements to a property such as improving condition or simply renovating only accounts for 2% in the change in price of a property. Minimal impact.

#### Question 3 - Are higher prices associated with larger properties, larger land or both?

In [ ]:
df_size.columns

In [ ]:
a = df_without_outlier_rows.loc[:,['sqft_living', 'sqft_lot', 'livingsqft_lotsqft_ratio']]

pd.plotting.scatter_matrix(a,figsize  = [15, 15]);
plt.show()

a.corr()

In [ ]:
# build model on 'living sqft' and 'lot sqft'

outcome = 'price'
predictors = df_size.drop(['price', 'livingsqft_lotsqft_ratio'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df_size).fit()
model.summary()

In [ ]:
# build model on 'living sqft - lot sqft ratio' only

outcome = 'price'
predictors = df_size.drop(['price', 'sqft_living', 'sqft_lot'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df_size).fit()
model.summary()

R-squared (**`sqft_living`**, **`sqft_lot`**) = **0.39** <br>
R-squared (**`livingsqft_lotsqft_ratio`**) = **0.04**

size of living space have a much greater association with price than the actual size of land

### BONUS REGRESSION

For the purpose of obtaining the highest R-squared possible

In [ ]:
# Only considering variables that exhibited some level of correlation with price

df_for_pairwise = df_without_outlier_rows.loc[:,['price', 'sqft_living',
       'sqft_lot',
       'sqft_above', 'sqft_basement',
       'sqft_living15', 'sqft_lot15', 'month',
       'dist_from_city_centre', 'livingsqft_lotsqft_ratio']]

In [ ]:
df_for_pairwise.corr()

In [ ]:
abs(df_for_pairwise.corr()) > 0.6

In [ ]:

sns.heatmap(df_for_pairwise.corr(), center=0);

In [ ]:
# excluding variables that showed high multicollinearity

df_final = df_without_outlier_rows.loc[:,['price', 'sqft_living',
       'sqft_lot',
       'month',
       'dist_from_city_centre',
        'd_bedro_2', 'd_bedro_3',
       'd_bedro_4', 'd_bedro_5', 'd_bedro_6', 'd_bathr_1',
       'd_bathr_2', 'd_bathr_3', 'd_bathr_4', 'd_bathr_5', 'd_floor_2', 'd_floor_3', 'd_water_1',
       'd_water_9', 'd_view_1', 'd_view_2', 'd_view_3', 'd_view_4',
       'd_condi_2', 'd_condi_3', 'd_condi_4', 'd_condi_5', 'd_decad_1910',
       'd_decad_1920', 'd_decad_1930', 'd_decad_1940', 'd_decad_1950',
       'd_decad_1960', 'd_decad_1970', 'd_decad_1980', 'd_decad_1990',
       'd_decad_2000', 'd_decad_2010', 'd_yrs_s_minus1', 'd_yrs_s_0',
       'd_yrs_s_10', 'd_yrs_s_20', 'd_yrs_s_30', 'd_yrs_s_40', 'd_yrs_s_50',
       'd_yrs_s_60', 'd_yrs_s_70', 'd_yrs_s_80']]



In [ ]:
outcome = 'price'
predictors = df_final.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df_final).fit()
model.summary()

R-squared = **0.63**

Visualisations for Presentation

In [ ]:
plt.figure(figsize=(10,10))
sqft_lot = sns.scatterplot(x="sqft_living", y="sqft_lot", hue = 'zipcode' , data=df_without_outlier_rows)